In [24]:
from batchgenerators.utilities.file_and_folder_operations import *
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
from skimage.transform import resize
from scipy.ndimage.interpolation import map_coordinates
from batchgenerators.augmentations.utils import resize_segmentation
from collections import OrderedDict

In [25]:
def resample_tindex(fold):
    
    vol_path = data_path+'/'+fold+'/imaging.nii.gz'
    seg_path = data_path+'/'+fold+'/segmentation_sinus.nii.gz'
    new_vol_path = data_path+'/'+fold+'/imaging_r.nii.gz'
    new_seg_path = data_path+'/'+fold+'/segmentation_sinus_r.nii.gz'
    
    if (not isfile(new_vol_path)) & (not isfile(new_seg_path)):
        a = nib.load(vol_path)
        vol = a.get_fdata()

        b = nib.load(seg_path)
        seg = b.get_fdata()

        aff = b.affine
        ori_x = abs(aff[0][0])
        ori_z = aff[2][2]
        ori_spacing = np.array([ori_x, ori_x, ori_z])
        new_spacing = np.array([ori_x, ori_x, ori_x])

        new_affine = aff.copy()
        new_affine[2][2] = ori_x

        vol = np.expand_dims(vol,axis=0)
        seg = np.expand_dims(seg,axis=0)
        vol_r, seg_r = resample_patient(vol, seg, ori_spacing, new_spacing, order_data=3, order_seg=1,
                                     force_separate_z=None, order_z_data=0, order_z_seg=0,
                                     separate_z_anisotropy_threshold=3)
        vol_r = np.squeeze(vol_r)
        seg_r = np.squeeze(seg_r)

        nifti = nib.Nifti1Image(vol_r, new_affine)
        nib.save(nifti, new_vol_path)
        nifti = nib.Nifti1Image(seg_r, new_affine)
        nib.save(nifti, new_seg_path)
        
    else:
        pass
    
    print('save complete: ', new_vol_path)
    print('save complete: ', new_seg_path)

In [26]:
def get_do_separate_z(spacing, anisotropy_threshold=3): # 3
    do_separate_z = (np.max(spacing) / np.min(spacing)) > anisotropy_threshold
    return do_separate_z
def get_lowres_axis(new_spacing):
    # 가장 높은 spacing값의 축(low resolution)
    axis = np.where(max(new_spacing) / np.array(new_spacing) == 1)[0]  # find which axis is anisotropic
    return axis

def resample_patient(data, seg, original_spacing, target_spacing, order_data=3, order_seg=0, force_separate_z=False,
                     cval_data=0, cval_seg=-1, order_z_data=0, order_z_seg=0,
                     separate_z_anisotropy_threshold=3):

    if data is not None:
        shape = np.array(data[0].shape)
    else:
        shape = np.array(seg[0].shape)
    new_shape = np.round(((np.array(original_spacing) / np.array(target_spacing)).astype(float) * shape)).astype(int)

    if force_separate_z is not None:
        do_separate_z = force_separate_z
        if force_separate_z:
            axis = get_lowres_axis(original_spacing)
        else:
            axis = None
    else:
        if get_do_separate_z(original_spacing, separate_z_anisotropy_threshold):
            do_separate_z = True
            axis = get_lowres_axis(original_spacing)
        elif get_do_separate_z(target_spacing, separate_z_anisotropy_threshold):
            do_separate_z = True
            axis = get_lowres_axis(target_spacing)
        else:
            do_separate_z = False
            axis = None

    if axis is not None:
        if len(axis) == 3:
            axis = (0, )
        elif len(axis) == 2:
            print("WARNING: axis has len 2, axis: %s, spacing: %s, target_spacing: %s" % (str(axis), original_spacing, target_spacing))
            do_separate_z = False
        else:
            pass
        
    if data is not None:
        data_reshaped = resample_data_or_seg(data, new_shape, False, axis, order_data, do_separate_z, cval=cval_data,
                                             order_z=order_z_data)
    else:
        data_reshaped = None
    if seg is not None:
        seg_reshaped = resample_data_or_seg(seg, new_shape, True, axis, order_seg, do_separate_z, cval=cval_seg,
                                            order_z=order_z_seg)
    else:
        seg_reshaped = None
    return data_reshaped, seg_reshaped

def resample_data_or_seg(data, new_shape, is_seg, axis=None, order=3, do_separate_z=False, cval=0, order_z=0):
    assert len(data.shape) == 4, "data must be (c, x, y, z)"
    if is_seg:
        resize_fn = resize_segmentation # batchgenerators
        kwargs = OrderedDict()
    else:
        resize_fn = resize # skimage
        kwargs = {'mode': 'edge', 'anti_aliasing': False}

    dtype_data = data.dtype
    data = data.astype(float)
    shape = np.array(data[0].shape)
    new_shape = np.array(new_shape)

    if np.any(shape != new_shape):
        if do_separate_z:
            print("separate z, order in z is", order_z, "order inplane is", order)
            assert len(axis) == 1, "only one anisotropic axis supported"
            axis = axis[0]
            if axis == 0: # [1,2]
                new_shape_2d = new_shape[1:]
            elif axis == 1: # [0,2]
                new_shape_2d = new_shape[[0, 2]]
            else: # [0,1]
                new_shape_2d = new_shape[:-1]

            reshaped_final_data = []
            for c in range(data.shape[0]):
                reshaped_data = []
                for slice_id in range(shape[axis]):
                    if axis == 0:
                        reshaped_data.append(resize_fn(data[c, slice_id], new_shape_2d, order, **kwargs))
                    elif axis == 1:
                        reshaped_data.append(resize_fn(data[c, :, slice_id], new_shape_2d, order, **kwargs))
                    else:
                        reshaped_data.append(resize_fn(data[c, :, :, slice_id], new_shape_2d, order, **kwargs))
                reshaped_data = np.stack(reshaped_data, axis)

                # 나머지 한 축 resampling (resize: scipy.ndimage.interpolation.map_coordinates)
                if shape[axis] != new_shape[axis]:
                    rows, cols, dim = new_shape[0], new_shape[1], new_shape[2]
                    orig_rows, orig_cols, orig_dim = reshaped_data.shape

                    row_scale = float(orig_rows) / rows
                    col_scale = float(orig_cols) / cols
                    dim_scale = float(orig_dim) / dim

                    map_rows, map_cols, map_dims = np.mgrid[:rows, :cols, :dim]
                    map_rows = row_scale * (map_rows + 0.5) - 0.5
                    map_cols = col_scale * (map_cols + 0.5) - 0.5
                    map_dims = dim_scale * (map_dims + 0.5) - 0.5

                    coord_map = np.array([map_rows, map_cols, map_dims])
                    if not is_seg or order_z == 0:
                        reshaped_final_data.append(map_coordinates(reshaped_data, coord_map, order=order_z, 
                                                                   mode='nearest')[None])
                    else:
                        unique_labels = np.unique(reshaped_data)
                        reshaped = np.zeros(new_shape, dtype=dtype_data)

                        for i, cl in enumerate(unique_labels):
                            reshaped_multihot = np.round(
                                map_coordinates((reshaped_data == cl).astype(float), coord_map, order=order_z,
                                                mode='nearest'))
                            reshaped[reshaped_multihot > 0.5] = cl
                        reshaped_final_data.append(reshaped[None])
                else:
                    reshaped_final_data.append(reshaped_data[None])
            reshaped_final_data = np.vstack(reshaped_final_data)

        else:
            print("no separate z, order", order)
            reshaped = []
            for c in range(data.shape[0]):
                reshaped.append(resize_fn(data[c], new_shape, order,  **kwargs)[None])
            reshaped_final_data = np.vstack(reshaped)
        return reshaped_final_data.astype(dtype_data)
    else:
        print("no resampling necessary")
        return data

In [4]:
#data_path =  '/home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_mask'
data_path =  '/home/ldh/1_Tindex/data/T_Index_nii'
folders = subfolders(data_path, join=False)

In [5]:
from multiprocessing import Pool
p = Pool(3)
p.map(resample_tindex, folders)
p.close()
p.join()

save complete: save complete: save complete:   

# Test

In [6]:
#data_path =  '/home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_pred'
folders = subfolders(data_path, join=False)

In [7]:
from multiprocessing import Pool
p = Pool(3)
p.map(resample_tindex, folders)
p.close()
p.join()

separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 1
save complete:  /home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_pred/313_7074817_151218_preop/imaging_r.nii.gz
save complete:  /home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_pred/313_7074817_151218_preop/segmentation_sinus_r.nii.gz
separate z, order in z is 0 order inplane is 3
save complete:  /home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_pred/336_3828394_160818_preop/imaging_r.nii.gz
save complete:  /home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_pred/336_3828394_160818_preop/segmentation_sinus_r.nii.gz
separate z, order in z is 0 order inplane is 3
save complete:  /home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_pred/364_8479256_170705_preop/imaging_r.nii.gz
sa

separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 1
save complete:  /home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_pred/433_8722184_180821_preop/imaging_r.nii.gz
save complete:  /home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_pred/433_8722184_180821_preop/segmentation_sinus_r.nii.gz
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
save complete:  /home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_pred/444_8758096_181030_preop_dual/imaging_r.nii.gz
save complete:  /home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_pred/444_8758096_181030_preop_dual/segmentation_sinus_r.nii.gz
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
save complete:  /home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_pred/434_8725519_180906_preop/imaging_r.nii.gz
save complete:  /home/ldh/1_Tindex/data

save complete:  /home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_pred/565_8849689_191129_preop/segmentation_sinus_r.nii.gz


In [27]:
def resample_tindex(fold):
    try:
        vol_path = data_path+'/'+fold+'/imaging.nii.gz'
        seg_path = data_path+'/'+fold+'/segmentation_sinus.nii.gz'
        new_vol_path = data_path+'/'+fold+'/imaging_r.nii.gz'
        new_seg_path = data_path+'/'+fold+'/segmentation_sinus_r.nii.gz'

        #if (not isfile(new_vol_path)) & (not isfile(new_seg_path)):
        a = nib.load(vol_path)
        vol = a.get_fdata()

        b = nib.load(seg_path)
        seg = b.get_fdata()

        aff = b.affine
        ori_x = abs(aff[0][0])
        ori_z = aff[2][2]
        ori_spacing = np.array([ori_x, ori_x, ori_z])
        new_spacing = np.array([ori_x, ori_x, ori_x])

        new_affine = aff.copy()
        new_affine[2][2] = ori_x

        vol = np.expand_dims(vol,axis=0)
        seg = np.expand_dims(seg,axis=0)
        vol_r, seg_r = resample_patient(vol, seg, ori_spacing, new_spacing, order_data=3, order_seg=1,
                                     force_separate_z=None, order_z_data=0, order_z_seg=0,
                                     separate_z_anisotropy_threshold=3)
        vol_r = np.squeeze(vol_r)
        seg_r = np.squeeze(seg_r)

        nifti = nib.Nifti1Image(vol_r, new_affine)
        nib.save(nifti, new_vol_path)
        nifti = nib.Nifti1Image(seg_r, new_affine)
        nib.save(nifti, new_seg_path)
    except:
        print(fold)
        
    #else:
    #    pass
    
    #print('save complete: ', new_vol_path)
    #print('save complete: ', new_seg_path)

In [19]:
data_path =  '/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_0'
for f in subfolders(data_path, join=False):
    resample_tindex(f)

separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, o

In [20]:
data_path =  '/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_1'
for f in subfolders(data_path, join=False):
    resample_tindex(f)

separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
004_10506248_211220_NP_preop_HK
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 

In [21]:
data_path =  '/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_2'
for f in subfolders(data_path, join=False):
    resample_tindex(f)

separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, o

In [22]:
data_path =  '/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_3'
for f in subfolders(data_path, join=False):
    resample_tindex(f)

separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, o

In [23]:
data_path =  '/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_4'
for f in subfolders(data_path, join=False):
    resample_tindex(f)

separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, o

In [28]:
data_path =  '/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_mask'
for f in subfolders(data_path, join=False):
    resample_tindex(f)

separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, o

In [29]:
data_path =  '/home/ldh/1_Tindex/data/T_Index_nii_test_A/T_Index_extract_mask'
for f in subfolders(data_path, join=False):
    resample_tindex(f)
data_path =  '/home/ldh/1_Tindex/data/T_Index_nii_test_A/T_Index_extract_pred'
for f in subfolders(data_path, join=False):
    resample_tindex(f)

no separate z, order 3
no separate z, order 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
no separate z, order 3
no separate z, order 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, ord

In [30]:
data_path =  '/home/ldh/1_Tindex/data/T_Index_nii_test_B/T_Index_extract_mask'
for f in subfolders(data_path, join=False):
    resample_tindex(f)
data_path =  '/home/ldh/1_Tindex/data/T_Index_nii_test_B/T_Index_extract_pred'
for f in subfolders(data_path, join=False):
    resample_tindex(f)

no separate z, order 3
no separate z, order 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
separate z, or

In [4]:
data_path =  '/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_0'
folders = subfolders(data_path, join=False)
from multiprocessing import Pool
p = Pool(3)
p.map(resample_tindex, folders)
p.close()
p.join()

separate z, order in z is 0 order inplane is 3
separate z, order in z isseparate z, order in z is  00  order inplane isorder inplane is  33

separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 1
separate z, order in z is 0 order inplane is 1
save complete:  /home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_0/001_2627046_211022_CMP_preop_KO/imaging_r.nii.gz
save complete:  /home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_0/001_2627046_211022_CMP_preop_KO/segmentation_sinus_r.nii.gz
separate z, order in z is 0 order inplane is 3
save complete:  /home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_0/002_3685594_211022_NP_preop_HK/imaging_r.nii.gz
save complete:  /home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_0/002_3685594_211022_NP_preop_HK/segmentation_sinus_r.nii.gz
save complete:  /home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_0/003_2597321_211217_CMP_pr

Process ForkPoolWorker-4:
Process ForkPoolWorker-3:
Process ForkPoolWorker-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ldh/anaconda3/envs/uk/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/ldh/anaconda3/envs/uk/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/ldh/anaconda3/envs/uk/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/ldh/anaconda3/envs/uk/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ldh/anaconda3/envs/uk/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ldh/anaconda3/envs/uk/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ldh/anaconda3/envs/uk/lib/

KeyboardInterrupt: 